# Data mining

## Modules

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from itertools import permutations
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from datasketch import MinHash
from tabulate import tabulate 
from scipy.sparse import csr_matrix
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

## Functionalities

### Plot routes, matrices and dataframes

In [ ]:
def plot_routes(df, key):
    """
    Plots the 2D and 3D vectors from the DataFrame with all possible axis combinations in 3D, including centroids.

    Args:
    df (pandas.DataFrame): The DataFrame to plot.
    key (str): The key in the DataFrame that contains the vectors to plot.
    """
    pca2 = PCA(n_components=2)
    pca3 = PCA(n_components=3)

    vectors = df[key].tolist()
    reduced_vectors_2 = pca2.fit_transform(vectors)

    if len(vectors[0]) != 3:
        vectors = pca3.fit_transform(vectors)

    # Create 7 subplots in a 4x2 layout (4 rows, 2 columns)
    fig = plt.figure(figsize=(20, 40))

    # 2D plotting
    ax2d = fig.add_subplot(4, 2, 1)  # 4 rows, 2 columns, position 1
    ax2d.scatter([vec[0] for vec in reduced_vectors_2], [vec[1] for vec in reduced_vectors_2], cmap='Spectral', alpha=0.1)
    ax2d.set_xlabel('PCA Component 1')
    ax2d.set_ylabel('PCA Component 2')
    ax2d.set_title(f'2D PCA Plot of {key}')

    # 3D plotting for each axis combination
    for i, (dim1, dim2, dim3) in enumerate(permutations(range(3), 3), start=3):
        ax3d = fig.add_subplot(4, 2, i, projection='3d')
        ax3d.scatter([vec[dim1] for vec in vectors], [vec[dim2] for vec in vectors], [vec[dim3] for vec in vectors], cmap='Spectral', depthshade=True, alpha=0.1)
        ax3d.set_xlabel(f'Component {dim1+1}')
        ax3d.set_ylabel(f'Component {dim2+1}')
        ax3d.set_zlabel(f'Component {dim3+1}')
        ax3d.set_title(f'3D Plot of Vectors ({dim1+1}, {dim2+1}, {dim3+1}) of {key}')

    plt.tight_layout()
    plt.show()

def plot_matrix(matrix, max_x=100, max_y=100, xlabel='X', ylabel='Y', title='No title'):
    """
    Plots a heatmap of the given matrix using seaborn.

    This function can plot the entire matrix or a specified part of it, depending on the max_x and max_y parameters.
    It configures the aesthetics of the plot including the size, color map, line color, and linewidths.

    Args:
    matrix (2D array-like): The matrix to be plotted.
    max_x (int, optional): The maximum number of rows to plot from the matrix. Defaults to 100.
    max_y (int, optional): The maximum number of columns to plot from the matrix. Defaults to 100.
    xlabel (str, optional): Label for the x-axis. Defaults to 'X'.
    ylabel (str, optional): Label for the y-axis. Defaults to 'Y'.
    title (str, optional): Title of the plot. Defaults to 'No title'.

    Returns:
    None: The function doesn't return anything; it only displays the plot.
    """
    resized_matrix = matrix[:max_x, :max_y] if max_x and max_y else matrix

    plt.figure(figsize=(20, 15))
    sns.heatmap(resized_matrix, annot=True, cmap='coolwarm', linecolor='white', linewidths=0.2)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

def printTable(df, title, showIndex=True, limit=10):
    """
    Prints a formatted table of the DataFrame to the console.

    This function converts any NumPy arrays in the DataFrame to string format for better readability.
    It uses the 'tabulate' library to format the table.

    Args:
    df (DataFrame): The DataFrame to be printed.
    title (str): The title of the table.
    showIndex (bool, optional): Flag to indicate if the index of the DataFrame should be shown. Defaults to True.
    limit (int, optional): The maximum number of rows to print from the DataFrame. Defaults to 10.

    Returns:
    None: The function doesn't return anything; it only prints the table to the console.
    """
    np.set_printoptions(threshold=np.inf)
    df_to_print = df.copy()
    for col in df.columns:
        if isinstance(df[col].iloc[0], np.ndarray):
            df_to_print[col] = df[col].apply(lambda x: np.array2string(x))

    print('\n', title)
    print(tabulate(df_to_print[:limit], headers='keys', showindex=showIndex, tablefmt='grid'))

    

def plot_vector_elements(df, key, cluster_column):
    """
    Plots the 2D and 3D vectors from the DataFrame with all possible axis combinations in 3D, including centroids.

    Args:
    df (pandas.DataFrame): The DataFrame to plot.
    key (str): The key in the DataFrame that contains the vectors to plot.
    cluster_column (str): The column name in the DataFrame that contains the cluster labels.
    """
    pca2 = PCA(n_components=2)
    pca3 = PCA(n_components=3)

    vectors = df[key].tolist()
    reduced_vectors_2 = pca2.fit_transform(vectors)

    if len(vectors[0]) != 3:
        vectors = pca3.fit_transform(vectors)

    # Create 7 subplots (1 for 2D and 6 for each axis combination in 3D)
    fig = plt.figure(figsize=(12, 60))

    # 2D plotting
    ax2d = fig.add_subplot(711)  # 7 total rows, 1 column, position 1
    ax2d.scatter([vec[0] for vec in reduced_vectors_2], [vec[1] for vec in reduced_vectors_2], c=df[cluster_column], cmap='Spectral', alpha=0.1)
    ax2d.set_xlabel('PCA Component 1')
    ax2d.set_ylabel('PCA Component 2')
    ax2d.set_title('2D PCA Plot of Vectors')

    # Calculate 2D centroids
    for cluster_label in df[cluster_column].unique():
        cluster_data_2d = reduced_vectors_2[df[cluster_column] == cluster_label]
        centroid_2d = cluster_data_2d.mean(axis=0)
        ax2d.scatter(*centroid_2d, color='red', marker='o', s=30, edgecolors='black', label='Centroid' if cluster_label == 0 else "")
        
    ax2d.legend()
    # 3D plotting for each axis combination
    for i, (dim1, dim2, dim3) in enumerate(permutations(range(3), 3), start=2):  # Start with subplot position 2
        ax3d = fig.add_subplot(7, 1, i, projection='3d')
        ax3d.scatter([vec[dim1] for vec in vectors], [vec[dim2] for vec in vectors], [vec[dim3] for vec in vectors], c=df[cluster_column], cmap='Spectral', depthshade=True, alpha=0.1)
        ax3d.set_xlabel(f'Component {dim1+1}')
        ax3d.set_ylabel(f'Component {dim2+1}')
        ax3d.set_zlabel(f'Component {dim3+1}')
        ax3d.set_title(f'3D Plot of Vectors ({dim1+1}, {dim2+1}, {dim3+1})')

        # Calculate and plot 3D centroids for each axis combination
        for cluster_label in df[cluster_column].unique():
            cluster_data_3d = np.array([vectors[i] for i, label in enumerate(df[cluster_column]) if label == cluster_label])
            centroid_3d = cluster_data_3d.mean(axis=0)
            ax3d.scatter(centroid_3d[dim1], centroid_3d[dim2], centroid_3d[dim3], color='red', marker='o', s=30, edgecolors='black', label='Centroid' if cluster_label == 0 else "")
        ax3d.legend()

    plt.tight_layout()
    plt.show()

def print_driver_top_routes_from_json(json_filename, standard_route_df):
    """
    Reads drivers data from a JSON file and prints each driver's top 5 routes.
    Function to inspect the point 2 of the project tasks.

    Args:
        json_filename (str): Path to the JSON file containing drivers' data.
        standard_route_df (pandas.DataFrame): DataFrame containing standard routes to match against.

    The function reads the JSON file, iterates through each driver, and prints their routes, matching them with the standard routes DataFrame.
    """

    with open(json_filename, 'r') as file:
        drivers_data = json.load(file)

    # Iterate through each driver in the JSON file
    for driver_data in drivers_data:
        print(f"Driver {driver_data['driver']}:")
        for route in driver_data['routes']:

            # Find the corresponding route in the standard routes DataFrame
            matching_route = standard_route_df[standard_route_df['sroute_id'] == route]
            if not matching_route.empty:

                route_description = matching_route.iloc[0]['route']
                print(f"{route} - {route_description}")
            else:
                print(f"{route} - No matching route found")

    return drivers_data

def plot_similarity(driver_values):

    transposed_data = driver_values.T.values
    names = driver_values.columns

    for i, line in enumerate(transposed_data, start=0):
        plt.plot(line, label=names[i])

    plt.xlabel('Drivers')
    plt.ylabel('Similarity (Non-divergence)')
    plt.legend()
    plt.show()

### Load routes

In [ ]:
def load_routes(route_path, vocab, encoder, route_type):
    """
    Loads routes from a file, processes them, and returns a DataFrame with encoded and additional route information.

    Args:
        route_path (str): Path to the JSON file containing route data.
        vocab (Vocabulary object): An instance of the Vocabulary class for encoding purposes.
        encoder (Encoder object): An encoder instance used for transforming route data.
        route_type (str): Type of the routes ('actual' or other).

    Returns:
        pandas.DataFrame: A DataFrame containing the processed and encoded route data.

    The function performs the following steps:
    1. Loads route data from a JSON file specified by `route_path`.
    2. Creates a DataFrame from the loaded data.
    3. Depending on the `route_type`, renames columns for clarity.
    4. Generates sequences of cities and trips from the route data using the encoder.
    5. Encodes the routes using the provided vocabulary.
    6. Separates the encoded data into matrices and vectors, vectors are the flatten matrices.
    7. Generates MinHash signatures for each route.
    8. Returns the DataFrame with all the processed and encoded route information.
    """

    print(f'Loading {route_type} routes... ')
    with open(route_path, 'r') as f:
        routes_data = json.load(f)

    routes_df = pd.DataFrame(routes_data)
    
    if route_type == "actual":
        # Renaming columns for actual routes
        routes_df.rename(columns={'id': 'aroute_id', 'sroute': 'sroute_id'}, inplace=True)
    else:
        # Renaming columns for other types of routes
        routes_df.rename(columns={'id': 'sroute_id'}, inplace=True)
    
    # Generate city sequence for each route
    routes_df['cities'] = routes_df['route'].apply(vocab.create_city_sequence)
    # Generate trip sequence for each route
    routes_df['trips'] = routes_df['route'].apply(vocab.create_trip_sequence)

    # Encode routes using the provided vocabulary
    routes_df['encoded_route'] = vocab.encode_route(routes_df)
    print(f' -> Routes encoded')

    # Separate encoded data into matrices and vectors
    # Using zip and * to unpack matrix and vector returned by encode_m1 & encode_m2
    routes_df['matrix'], routes_df['vector'] = zip(*routes_df['encoded_route'].apply(lambda r: encoder.vectorize(r)))
    print(' -> Matrices & vectors generated.')

    # Generate MinHash signatures for each route
    routes_df['signature'] = routes_df['matrix'].apply(encoder.minhash_signature)
    print(' -> Minhash signatures generated.')
    
    print(f'Loaded {route_type} routes.\n')
    return routes_df

def load_driver_ideal_routes(route_path, vocabulary, encoder):
    """
    Loads optimal route data from a file, processes them, and returns a DataFrame with encoded and additional route information.

    Args:
        route_path (str): Path to the JSON file containing route data.
        vocabulary (Vocabulary object): An instance of the Vocabulary class for encoding purposes.
        encoder (Encoder object): An encoder instance used for transforming route data.

    Returns:
        pandas.DataFrame: A DataFrame containing the processed and encoded route data.

    The function performs the same steps of the above function and return a dataframe with the same structure.
    """

    print(f'Loading optimal routes... ')
    with open(route_path, 'r') as f:
        routes_data = json.load(f)

    routes_df = pd.DataFrame(routes_data)

    routes_df.rename(columns={'id': 'aroute_id'}, inplace=True)
    
    # Generate city sequence for each route
    routes_df['cities'] = routes_df['route'].apply(encoder.create_city_sequence)
    # Generate trip sequence for each route
    routes_df['trips'] = routes_df['route'].apply(encoder.create_trip_sequence)

    # Encode routes using the provided vocabulary
    routes_df['encoded_route'] = vocabulary.encode_route(routes_df)
    print(f' -> Routes encoded')

    # Separate encoded data into matrices and vectors
    # Using zip and * to unpack matrix and vector returned by encode_m1 & encode_m2
    routes_df['matrix'], routes_df['vector'] = zip(*routes_df['encoded_route'].apply(lambda r: encoder.vectorize(r)))
    print(' -> Matrices & vectors generated.')

    # Generate MinHash signatures for each route
    routes_df['signature'] = routes_df['matrix'].apply(encoder.minhash_signature)
    print(' -> Minhash signatures generated.')
    
    print(f'Loaded optimal routes.\n')

    return routes_df

### Save routes

In [ ]:
def save_top_routes_to_json(df, filename):
    """
    Saves the top routes of drivers to a JSON file.

    Args:
        df (pandas.DataFrame): DataFrame containing drivers and their top routes.
        filename (str): Name of the file to save the data.

    The function iterates through each driver in the DataFrame, extracting their top 5 routes, and then saves this data to a JSON file.
    """
    drivers_data = []

    for driver, row in df.iterrows():
        top_routes_df = row['top_routes']

        # Extract top 5 routes from the nested DataFrame
        top_routes = top_routes_df.index[:5].tolist()
        drivers_data.append({'driver': driver, 'routes': top_routes})

    with open(filename, 'w') as file:
        json.dump(drivers_data, file, indent=4)
 
def save_standard_routes(df, column_name, file_path):
    """
    Saves standard routes from a DataFrame to a JSON file in the standard format.

    Args:
        df (pandas.DataFrame): DataFrame containing route data.
        column_name (str): Column name in the DataFrame that contains route IDs.
        file_path (str): Path to save the JSON file.

    The function creates a list of dictionaries from the DataFrame, where each dictionary contains route ID and details, and then saves it to a JSON file.
    """
    data_to_save = [
        {"id": row[column_name], "route": row[column_name]} 
        for index, row in df.iterrows()
    ]

    with open(file_path, 'w') as file:
        json.dump(data_to_save, file, indent=2)

def save_new_standard_routes(data_to_save, file_path):
    """
    Saves new standard routes to a JSON file.

    Args:
        data_to_save (list of dicts): Data containing route information to be saved.
        file_path (str): Path to the JSON file where the data will be saved.

    The function writes the route data to a JSON file, using a custom converter for data types that are not natively serializable by JSON, such as numpy int32.
    """    
    def default_converter(o):
        if isinstance(o, np.int32):
            return int(o)
        if isinstance(o, np.int64):
            return int(o)
        raise TypeError(f'Object of type {o.__class__.__name__} is not JSON serializable')

    with open(file_path, 'w') as file:
        json.dump(data_to_save, file, indent=2, default=default_converter)

### Encoder

The encoder class contains methods to:
- Generate a vector from each route.
- Extract the cities or trips sequence from a route
- Apply PCA to a specified list of vectors.

In [ ]:
class Encoder:
    def __init__(self, vocabulary, num_permutation):
        """
        Initializes the Encoder class with a vocabulary and a number of permutations for MinHash.

        Args:
            vocabulary (Vocabulary object): An instance of the Vocabulary class containing mappings for cities and merchandise.
            num_permutation (int): The number of permutations to use for MinHash signatures.
        """
        self.num_permutation = num_permutation
        self.vocabulary = vocabulary
        
    def vectorize(self, encoded_route):
        """
        Creates a matrix and vector representation of a route based on city and merchandise information.

        Args:
            encoded_route (list of tuples): Each tuple contains encoded city and merchandise data for a trip.

        Returns:
            tuple: A tuple containing a matrix representation of the route and a flattened vector form of the same.
            
        The function constructs a matrix where each row represents a trip. The row is a concatenation of a city vector and 
        a normalized merchandise vector. The city vector has an incremental value based on order at the indices representing the start and end cities of the trip, 
        and the merchandise vector has quantities for each merchandise type.
        """
        cities_vector_length = len(self.vocabulary.city2id)
        merch_vector_length = len(self.vocabulary.merchandise2id)
        max_trip_count = self.vocabulary.max_trip

        route_matrix = np.zeros((max_trip_count, cities_vector_length + merch_vector_length))
        
        for trip_order, (trip, merch) in enumerate(encoded_route):       

            cities_vector = np.zeros(cities_vector_length)
            cities_vector[trip[0]] = 2 * trip_order + 1
            cities_vector[trip[1]] = 2 * trip_order + 2

            merch_vector = np.zeros(merch_vector_length)
            for merch_id, qty in merch.items():
                merch_vector[merch_id] = qty

            normalized_merch_vector = merch_vector / (self.vocabulary.max_trip * 2) # ---> Merchandise vector is normalized using the maximum quantity of merch in the dataset.
            cities_merch_vector = np.concatenate([cities_vector, normalized_merch_vector])
            
            route_matrix[trip_order, :] = cities_merch_vector

        route_vector = np.array(route_matrix.flatten())

        return route_matrix, route_vector

    def apply_PCA(self, df, key, n_components = 10, suffix = '_pca'):
        """
        Applies Principal Component Analysis (PCA) to reduce the dimensionality of data.

        Args:
            df (pandas.DataFrame): DataFrame containing the data.
            key (str): Column name in the DataFrame which contains the data for PCA.
            n_components (int, optional): Number of principal components to keep.
            suffix (str, optional): Suffix to append to new PCA columns. Defaults to '_pca'.

        This method reduces the dimensionality of the data in 'key' column of DataFrame 'df' using PCA and adds the 
        results as new columns in the DataFrame.
        """
        vectors = df[key].tolist()
        pca = PCA(n_components=n_components)
        pca_results = pca.fit_transform(vectors)

        for i in range(n_components):
            df[f'PCA{i}'] = np.array(pca_results[:, i])

        df[f'{key}{suffix}'] = df.apply(lambda row: np.array([row[f'PCA{i}'] for i in range(n_components)]), axis=1)

    def minhash_signature(self, vector):
        """
        Computes the MinHash signature of a given vector.

        Args:
            vector (numpy.ndarray): The vector to compute the MinHash signature for.

        Returns:
            list: A list of MinHash signatures.

        This method computes a MinHash signature for the input vector using the specified number of permutations.
        """
        minhash = MinHash(num_perm=self.num_permutation)
        minhash.update(vector.tobytes())
        return list(minhash.hashvalues)


### Vocabulary


In [ ]:
class Vocabulary:

    def __init__(self, standard_route_path, actual_route_path):
        """
        Initializes the Vocabulary class by loading and processing route data from JSON files.

        Args:
            standard_route_path (str): Path to the JSON file containing standard routes.
            actual_route_path (str): Path to the JSON file containing actual routes.

        Attributes:
            max_trip (int): The maximum number of trips in a route found in the data.
            min_qty (float): The minimum quantity of any merchandise item across all routes.
            max_qty (float): The maximum quantity of any merchandise item across all routes.
            cities, merchandise, trips_city_pair, drivers, stdroutes, actroutes (set): Sets to store unique entities.
            city2id, id2city, merchandise2id, id2merchandise, etc. (dict): Dictionaries for entity to ID mappings.
        """

        # Initialize maximum, minimum and data sets
        self.max_trip = 0
        self.min_qty = float('inf')
        self.max_qty = 0

        # Data sets
        self.cities = set()
        self.merchandise = set()
        self.trips_city_pair = set()
        self.drivers = set()
        self.stdroutes = set()
        self.actroutes = set()

        # Process routing data from JSON files
        self.process_routes(actual_route_path, self.actroutes)
        self.process_routes(standard_route_path, self.stdroutes)

        # Create vocabularies (mappings)
        self.city2id, self.id2city = self.create_vocab(self.cities)
        self.merchandise2id, self.id2merchandise = self.create_vocab(self.merchandise)
        self.trip_city_pair2id, self.id2trip_city_pair = self.create_vocab(self.trips_city_pair)
        self.driver2id, self.id2driver = self.create_vocab(self.drivers)

        self.trips_set = set()
        self.process_trips(actual_route_path)
        self.process_trips(standard_route_path)

        self.trip2id, self.id2trip = self.create_vocab(self.trips_set)
        
    def process_routes(self, file_path, routes):
        """
        Processes routing data from a JSON file. Updates sets of cities, merchandise, trips, drivers, and routes.

        Args:
            file_path (str): Path to the JSON file to be processed.
            routes (set): A set to store unique route identifiers.

        This method reads route data from the specified file and updates various sets with unique items, like cities and merchandise.
        """
        try:
            with open(file_path, 'r') as f:
                data = pd.DataFrame(json.load(f))

            for _, row in data.iterrows():
                routes.add(row['id'])

                if 'driver' in row:
                    self.drivers.add(row['driver'])

                for i, trip in enumerate(row['route'], start=1):

                    self.cities.update([trip['from'], trip['to']])
                    self.trips_city_pair.add((trip['from'], trip['to']))

                    for item, qty in trip['merchandise'].items():

                        self.merchandise.add(item)
                        self.min_qty = min(self.min_qty, qty)
                        self.max_qty = max(self.max_qty, qty)

                    self.max_trip = max(self.max_trip, i)
        except IOError:
            print(f"Error reading file: {file_path}")

    def process_trips(self, file_path):
        """
        Processes trip data from a JSON file. Updates the set of unique trips.

        Args:
            file_path (str): Path to the JSON file to be processed.

        This method reads trip data from each route of the specified file and updates the set of unique trips.
        """
        try:
            with open(file_path, 'r') as f:
                data = pd.DataFrame(json.load(f))

            for _, row in data.iterrows():
                for _, trip in enumerate(row['route'], start=1):
                    merch = []
                    for item, qty in trip['merchandise'].items():
                        merch.append((item, qty))

                    element = [trip['from']] + [trip['to']] + merch
                    self.trips_set.add(tuple(element))

        except IOError:
            print(f"Error reading file: {file_path}")

    def create_vocab(self, data):
        """
        Creates a vocabulary for a given set of data.

        Args:
            data (iterable): A collection of unique items.

        Returns:
            tuple: Two dictionaries, one mapping items to IDs and the other mapping IDs back to items.

        This method generates a vocabulary for encoding and decoding purposes.
        """
        vocab = {el: i for i, el in enumerate(data)}
        inverse_vocab = {i: el for el, i in vocab.items()}
        return vocab, inverse_vocab
    
    def encode_route(self, dataframe):
        """
        Encodes routes in a DataFrame using the established vocabularies.

        Args:
            dataframe (pandas.DataFrame): DataFrame containing route data.

        Returns:
            list: A list where each item is an encoded representation of a route.

        This method encodes the routes in the DataFrame, converting city and merchandise information into their corresponding IDs.
        """
        encoded_routes = []
        for _, row in dataframe.iterrows():
            route = row['route']
            encoded_route = []

            for trip in route:
                start_city = trip['from']
                end_city = trip['to']
                merchandises = trip['merchandise']

                encoded_merch = {self.merchandise2id[merch] : qty for merch, qty in merchandises.items()}
                encoded_cities = (self.city2id.get(start_city), self.city2id.get(end_city))
                
                encoded_trip = (encoded_cities, encoded_merch)
                encoded_route.append(encoded_trip)

            encoded_routes.append(encoded_route)

        return encoded_routes

    def create_city_sequence(self, route):
        """
        Converts a route into a sequence of cities.

        Args:
            route (list of dicts): A route represented as a list of dictionaries, each dict containing 'from' and 'to' keys.

        Returns:
            list: A sequential list of cities in the order they are visited in the route.

        This method creates a list of cities visited in a route based on the 'from' and 'to' keys of each trip in the input route.
        """
        taken_route = []
        for path in route:
            if taken_route:
                taken_route.append(path['to'])
            else:
                taken_route.append(path['from'])
                taken_route.append(path['to'])
        return taken_route

    def create_trip_sequence(self, route):
        """
        Converts a route into a sequence of trips in a hashable format. Each trip is represented as a tuple.

        Args:
            route (list of dicts): A route

        Returns:
            list: A list of tuples, each representing a trip in the routeù

        This method iterates over each trip in the provided route. For each trip, it constructs a tuple that starts with 
        the origin ('from') and destination ('to') cities, followed by a list of (merchandise, quantity) pairs. This tuple 
        format encapsulates all the relevant information for a trip in a single, hashable data structure.
        """
        trips = []
        for trip in route:
            trips.append(tuple(
                [trip['from']] + 
                [trip['to']] +
                [(merch, qty) for merch, qty in trip['merchandise'].items()]
            ))
        return trips

### Similarity matrices generation

In [ ]:
def calculate_similarity_matrix(standard_routes_df, actual_routes_df, key_field, similarity, save_path):
    """
    Computes a similarity matrix between standard and actual routes based on the specified similarity measure.

    Args:
        standard_routes_df (pandas.DataFrame): DataFrame containing standard route data.
        actual_routes_df (pandas.DataFrame): DataFrame containing actual route data.
        key_field (str): Field name in both DataFrames used for similarity calculation.
        similarity (str): Type of similarity measure ('jaccard', 'cosine', or 'euclidean').
        save_path (str): Path to save the calculated similarity matrix.

    Returns:
        numpy.ndarray: A matrix where each element represents the similarity score between a pair of standard and actual routes.

    The function computes pairwise similarities for each actual route against all standard routes using the specified similarity measure.
    """

    # Load an existing similarity matrix if available
    similarity_matrix = load_similarity_matrix(save_path)
    if similarity_matrix is not None:
        print(f'Matrix loaded from file {save_path}.')
        return similarity_matrix

    # Initialize an empty similarity matrix
    num_actual_routes = len(actual_routes_df)
    num_standard_routes = len(standard_routes_df)
    similarity_matrix = np.zeros((num_actual_routes, num_standard_routes))

    # Compute the similarity for each pair of actual and standard routes
    for i, actual_row in tqdm(enumerate(actual_routes_df.itertuples()), total=num_actual_routes, desc='Building similarity matrix...'):
        for j, standard_row in enumerate(standard_routes_df.itertuples()):
            # Calculate similarity based on the chosen method
            if similarity == 'jaccard':
                similarity_matrix[i, j] = jaccard_similarity(getattr(actual_row, key_field), getattr(standard_row, key_field))
            elif similarity == 'cosine':
                similarity_matrix[i, j] = cosine_similarity(getattr(actual_row, key_field).reshape(1, -1), getattr(standard_row, key_field).reshape(1, -1))[0,0]

    # Save the calculated similarity matrix
    np.save(save_path, similarity_matrix)
    return similarity_matrix

def jaccard_similarity(signature1, signature2):
    """
    Computes the Jaccard similarity between two signatures.

    Args:
        signature1 (numpy.ndarray): The first signature array.
        signature2 (numpy.ndarray): The second signature array.

    Returns:
        float: The Jaccard similarity score between the two signatures.

    The Jaccard similarity is calculated as the sum of the minimum intersection divided by the sum of the maximum union of the two signatures.
    """
    min_intersection = np.minimum(signature1, signature2).sum()
    max_union = np.maximum(signature1, signature2).sum()

    # Handle division by zero
    if max_union == 0:
        return 0

    return min_intersection / max_union


def euclidean_similarity(vector1, vector2):
    """
    Calculates Euclidean similarity between two vectors.

    Args:
        vector1 (numpy.ndarray): The first vector.
        vector2 (numpy.ndarray): The second vector.

    Returns:
        float: The Euclidean similarity score between the two vectors.

    The Euclidean similarity is computed as an exponentially decreasing function of the Euclidean distance between the vectors.
    """
    # Compute the Euclidean distance
    distance = np.linalg.norm(vector1 - vector2)

    # Convert distance to similarity in the range [0, 1]
    similarity = np.exp(-distance)

    return similarity

def euclidean_similarity_matrix(X):
    """
    Calculates similarity based on Euclidean distance between rows of a matrix.

    Args:
        X (array-like): An input matrix where each row represents an item (e.g., a trip or a driver).

    Returns:
        numpy.ndarray: A similarity matrix where the element (i, j) represents the similarity between the i-th and j-th rows of X.

    This method computes the Euclidean distance between every pair of rows in X.
    To convert it into a measure of similarity, we use the formula 1 / (1 + distance). 1 is added to the denominator to avoid division by zero.
    """
    distances = euclidean_distances(X)
    # Transforming the distance into a measure of similarity
    similarity = 1 / (1 + distances)
    return similarity


def load_similarity_matrix(save_path):
    """
    Attempts to load a pre-existing similarity matrix from a specified file path.

    Args:
        save_path (str): Path where the similarity matrix file is expected to be located.

    Returns:
        numpy.ndarray or None: The similarity matrix if the file exists and is successfully loaded, otherwise None.

    This function is used to check for and load an existing similarity matrix, which can save computation time if the matrix has been previously calculated.
    """
    try:
        # Load the matrix from the specified path
        similarity_matrix = np.load(save_path)
        return similarity_matrix
    except FileNotFoundError:
        # Return None if the file does not exist
        return None


### Utility matrix

In [ ]:
class UtilityMatrix:
    
    def __init__(self, routes, comparison_key, vocabulary):
        """
        Initializes the UtilityMatrix class for creating a utility matrix from route data.

        Args:
            comparison_key (str): Key used for comparison in generating the utility matrix.
            vocabulary (Vocabulary object): An instance of the Vocabulary class.
            metric (str): label used to select the similarity metric to use for all the computations.

        Attributes:
            matrix_lv1, trip_count_matrix, trip_sum_matrix (numpy.ndarray): Matrices representing different levels of utility based on the routes.
            trip_count_matrix_norm (numpy.ndarray): Normalized trip count matrix.

        The class calculates utility matrices based on driver and trip data, using the provided vocabulary for encoding.
        """
        self.vocabulary = vocabulary
        self.key = comparison_key
        self.matrix_lv1, self.trip_count_matrix, self.trip_sum_matrix = self.driver_x_trip_matrix(routes)

    def driver_x_trip_matrix(self, routes):
        """
        Generates a utility matrix representing the relationship between drivers and trips.

        Args:
            routes (pandas.DataFrame): DataFrame containing route data.

        Returns:
            tuple: A tuple containing the computed utility matrix, trip count matrix, and trip sum matrix.

        This method calculates matrices that map the relationship between drivers and trips based on the trips data. 
        It uses th specified metric to measure the similarity between each actual trip of the actual route and each standard trip of the correspondant standard route.
        """
        # Initialize matrices for sum and count
        sum_matrix = np.zeros((len(self.vocabulary.driver2id), len(self.vocabulary.trip2id)))
        count_matrix = np.zeros((len(self.vocabulary.driver2id), len(self.vocabulary.trip2id)))

        # Calculate the length of encoded city and merchandise vectors
        city_encoded_length = len(self.vocabulary.city2id)
        merch_encoded_length = len(self.vocabulary.merchandise2id)
        trip_vector_len = city_encoded_length + merch_encoded_length       

        # Iterate through routes to populate matrices
        for _, row in tqdm(routes.iterrows(), total=len(routes), desc='Generating level 1 utility matrix'):
            driver_idx = self.vocabulary.driver2id.get(row['driver'])
            std_trips = row['trips_std']
            act_trips = row['trips_act']
            std_vector = row[f'{self.key}_std']
            act_vector = row[f'{self.key}_act']
        
            for i, act_trip_value in enumerate(act_trips):
                act_trip_index = self.vocabulary.trip2id.get(act_trip_value)
                cumulated_similarity = 0
                comparison_count = 0

                # Compare actual trip with standard trips using cosine similarity
                for k in range(len(std_trips)):

                    std_vector_slice = std_vector[k * trip_vector_len : (k+1) * trip_vector_len]
                    act_vector_slice = act_vector[i * trip_vector_len : (i+1) * trip_vector_len]

                    similarity_increase = cosine_similarity(act_vector_slice.reshape(1, -1), std_vector_slice.reshape(1, -1))[0, 0]

                    # Special handling when actual trip and standard trip are the same
                    if similarity_increase == 1 and i == k:
                        cumulated_similarity = 1
                        comparison_count = 1
                        break

                    cumulated_similarity += similarity_increase
                    comparison_count += 1

                # Update the sum and count matrices
                sum_matrix[driver_idx, act_trip_index] += cumulated_similarity
                count_matrix[driver_idx, act_trip_index] += comparison_count

        # Compute and return the utility matrices
        mask = count_matrix != 0
        um = np.zeros_like(count_matrix, dtype=float)
        um[mask] = np.round(sum_matrix[mask] / count_matrix[mask], 5)

        # Normalize the trip count matrix. This step is essential to compare trips on a similar scale later,
        # especially when the number of trips can vary significantly. Normalization is done by dividing 
        # each element in the trip count matrix by the maximum value in this matrix, resulting in a matrix 
        # where all count values are between 0 and 1.

        count_matrix = count_matrix / np.max(count_matrix)
        return um, count_matrix, sum_matrix


    def predict_ratings_with_driver_profile_top_k(self, driver_profiles, top_k_drivers, precision=5):
        """
        Predicts utility matrix ratings using driver profile similarities for the top-k similar drivers.

        Args:
            driver_profiles (numpy.ndarray): Matrix containing driver profile similarities.
            top_k_drivers (list): List of top-k similar drivers for each driver.
            precision (int): Decimal precision for the predicted ratings.

        Updates the matrix_lv2_driver attribute with the predicted ratings.
        """
        num_drivers, num_trips = self.matrix_lv1.shape
        predicted_matrix = self.matrix_lv1.copy()

        for driver_A in tqdm(range(num_drivers), total=num_drivers, desc='Predicting ratings with driver profiles'):
            # Use top-k similar drivers for prediction
            similar_drivers = top_k_drivers[driver_A]

            for trip in range(num_trips):
                if predicted_matrix[driver_A, trip] == 0:
                    # Calculate the weighted average rating
                    total_rating = sum(driver_profiles[driver_A, driver_B] * self.matrix_lv1[driver_B, trip] for driver_B in similar_drivers if self.matrix_lv1[driver_B, trip] > 0)
                    total_similarity = sum(driver_profiles[driver_A, driver_B] for driver_B in similar_drivers if self.matrix_lv1[driver_B, trip] > 0)

                    if total_similarity > 0:
                        predicted_rating = total_rating / total_similarity
                        predicted_matrix[driver_A, trip] = np.round(predicted_rating, precision)

        self.matrix_lv2 = predicted_matrix

        
def get_top_k_similar_indices(similarity_matrix, k):
    """
    Identifies the top-k similar indices for each row in a similarity matrix.
    Used with drivers and trips profiles.

    Args:
        similarity_matrix (numpy.ndarray): A square matrix where each element represents a similarity score.
        k (int): The number of top similar elements to identify for each row.

    Returns:
        dict: A dictionary where each key is a row index and the value is an array of indices of the top-k similar elements.

    This function computes the top-k similar indices for each row in the similarity matrix.
    It first converts the similarity matrix into a sparse matrix for efficient computation, then iterates over each row to find the top-k similar elements.
    """
    # Convert to sparse matrix for efficient computation
    sparse_sim = csr_matrix(similarity_matrix)
    top_k_indices = {}

    for index in tqdm(range(sparse_sim.shape[0]), total=sparse_sim.shape[0], desc=f'Computing top-{k} similarities'):
        row = sparse_sim[index]
        
        if row.nnz > k:
            # Indices of top-k highest values in the row
            top_indices = np.argsort(row.data)[-k:][::-1]
            top_k_indices[index] = row.indices[top_indices]
        else:
            # All indices if the non-zero elements are fewer than k
            top_k_indices[index] = row.indices

    return top_k_indices

### Profiles

In [ ]:
class DriversProfile:
    def __init__(self, utility_matrix, vocabulary):
        """
        Initializes the profiles class.

        Args:
            utility_matrix (numpy.ndarray): The utility matrix used for creating profiles.
            vocabulary (Vocabulary object): The vocabulary used in the utility matrix.
            metric (str): The metric to use to compute the similarity.
        Attributes:
            utility_matrix (numpy.ndarray): Stored utility matrix.
            vocabulary (Vocabulary object): Stored vocabulary object.
        """
        self.utility_matrix = utility_matrix
        self.vocabulary = vocabulary
        self.generate_driver_profiles(precision=5)

    def generate_driver_profiles(self, precision=5):
        """
        Generates cosine or Euclidean similarity profiles for each driver based on a reduced dimensionality
        of the utility matrix, achieved through PCA, reducing the feature space to 3 components.

        Args:
            precision (int): The decimal precision to use for the similarity scores.

        Updates:
            self.drivers_profile (numpy.ndarray): A matrix containing the cosine similarity scores between all pairs of drivers,
            post-dimensionality reduction.

        This method applies PCA on the utility matrix to reduce its dimensionality to 3 components before calculating
        the similarity. The resulting drivers_profile matrix provides a measure of similarity
        between drivers, indicating how similar two drivers are in terms of their utility patterns in this reduced feature space.
        """
        # Apply PCA to reduce dimensions to 3
        pca = PCA(n_components=10)
        reduced_matrix = pca.fit_transform(self.utility_matrix)

        # Calculate similarity between drivers based on the selected metric
        self.drivers_profile = np.round(cosine_similarity(reduced_matrix), precision)




### Feature generation

In [ ]:
def trip_evaluation(utility_matrix, weights, vocabulary):
    """
    Evaluates trips based on a utility matrix and weights, and ranks them accordingly.

    Args:
        utility_matrix (numpy.ndarray): The utility matrix representing trips.
        weights (numpy.ndarray): The weights associated with each trip.
        vocabulary (Vocabulary object): The vocabulary object containing trip IDs and other details.

    Returns:
        pandas.DataFrame: A DataFrame containing trip evaluation scores and additional information.

    This function computes various metrics for trips such as average score, count, and normalized scores. These metrics are used to rank the trips.
    """
    # Calculate the sum of non-zero elements in each column
    sum_non_zero_trips = np.where(utility_matrix != 0, utility_matrix, 0).sum(axis=0)
    
    # Count the number of non-zero values per column
    count_non_zero_trips = (utility_matrix != 0).sum(axis=0)

    # Calculate the average, avoiding division by zero
    trip_scores = np.divide(sum_non_zero_trips, count_non_zero_trips, out=np.zeros_like(sum_non_zero_trips))

    # Calculate the total counts and normalize them
    counts = weights.sum(axis=0)  # Get the total number of times that a trip has been executed
    normalized_counts = counts / np.max(counts)
    score = trip_scores * normalized_counts
    norm_score = score / score.max()

    # Create a DataFrame with trip evaluation details
    df = pd.DataFrame({
        'trip_id': range(len(vocabulary.trip2id)),
        'trip': [vocabulary.id2trip.get(i) for i in range(len(vocabulary.trip2id))],
        'trip_liked_avg_score': trip_scores,
        'count': counts,
        'normalized_count': normalized_counts,
        'score': score,
        'normalized_score' : norm_score
    })
    df.set_index('trip_id', inplace=True)
    return df.sort_values(by='normalized_score', ascending=False)


def add_trip_level_feature(trip_ranking, routes_df, vocabulary, feature_name):
    """
    Adds a trip-level feature to a DataFrame based on the trip rankings.

    Args:
        trip_ranking (pandas.DataFrame): DataFrame containing trip rankings and scores.
        routes_df (pandas.DataFrame): DataFrame to which the trip feature is to be added.
        vocabulary (Vocabulary object): The vocabulary object containing trip IDs and other details.
        feature_name (str): Name of the feature to be added.

    The function iterates through each row in the routes DataFrame, adds a new trip level feature based on the trip rankings, 
    and updates the DataFrame with new matrices and vectors with the correspondant feature value incorporated.
    """
    # New feature fields.
    routes_df['matrix_f1'] = None
    routes_df['vector_f1'] = None

    # Insepct each row of the dataframe
    for index, row in tqdm(routes_df.iterrows(), total = len(routes_df), desc = 'Adding trip rating feature'):


        preferences_features = np.zeros((vocabulary.max_trip, 1)) # Column of ne features to add to the old matrix
        trips = [vocabulary.trip2id.get(trip) for trip in row['trips']] # Get trip ids of the current row (route)

        for i, trip_id in enumerate(trips): # For each trip

            # Get the score from the input dataframe
            trip_score = trip_ranking.loc[trip_id, feature_name]

            # Set the score as feature in the column
            preferences_features[i, 0] = trip_score

        # Check shape
        assert row['matrix'].shape[0] == preferences_features.shape[0]

        # Add the column to the specified key field of the row
        combined_matrix = np.hstack((row['matrix'], preferences_features))
        
        # Insert the new objects into the original dataframe as new fields

        routes_df.at[index, 'matrix_f1'] = combined_matrix
        routes_df.at[index, 'vector_f1'] = combined_matrix.flatten()

### Clustering

In [ ]:
def cluster_and_evaluate(data, params, key, approach='kmeans'):
    """
    Performs clustering on the dataset and evaluates the clustering performance.

    This function applies a clustering algorithm (either KMeans or KMedoids) to the dataset based on the specified parameters and key. 
    It calculates and returns the Silhouette and Davies-Bouldin scores for the clustering, which are metrics used to evaluate the quality of the clustering.

    Args:
        data (pandas.DataFrame): The DataFrame containing the data to cluster.
        params (dict): Parameters for the clustering algorithm.
        key (str): The key in the DataFrame that contains the vectors to use for clustering.
        approach (str): The clustering approach to use ('kmeans' or 'kmedoids').

    Returns:
        tuple: A tuple containing the clustering model, Silhouette score, and Davies-Bouldin score.
    """
    # Select the clustering model based on the specified approach
    if approach == 'kmeans':
        model = KMeans(**params, random_state=1)
    else:
        model = KMedoids(**params, random_state=1,)

    # Fit the model to the data
    model.fit(data[key].tolist())
    # Assign cluster labels to the data
    data[f'cluster_{approach}'] = model.labels_

    # Calculate clustering evaluation metrics
    silhouette = silhouette_score(data[key].tolist(), model.labels_, metric='cosine')
    bouldin = davies_bouldin_score(data[key].tolist(), model.labels_)

    return model, silhouette, bouldin

def clustering_summary(df, cluster_column):
    """
    Generates a summary of each cluster.

    This function calculates the number of elements in each cluster and compiles a list of route IDs
    that belong to each cluster. It also plots a histogram showing the distribution of the clusters.

    Args:
        df (pandas.DataFrame): The DataFrame containing cluster labels and route IDs.
        cluster_column (str): The column name in the DataFrame that contains the cluster labels.

    Returns:
        pandas.DataFrame: A DataFrame containing a summary of each cluster, including the number of elements and the route IDs.
    """
    cluster_summary = []

    # Iterate through each unique cluster label
    for cluster_label in df[cluster_column].unique():
        cluster_items = df[df[cluster_column] == cluster_label]

        # Count the number of items and gather route IDs in the cluster
        num_items = len(cluster_items)
        route_ids = cluster_items['aroute_id'].tolist()  # or 'sroute_id' for standard routes

        # Append the cluster summary information
        cluster_summary.append({
            'cluster_label': cluster_label,
            'num_elements': num_items,
            'route_ids': route_ids,
        })

    # Convert the summary to a DataFrame
    summary_df = pd.DataFrame(cluster_summary)
    
    # Plot a histogram to visualize the distribution of clusters
    plt.figure(figsize=(12, 8))
    df[cluster_column].value_counts().sort_index().plot(kind='bar', color='skyblue')
    plt.xlabel('Cluster Label')
    plt.ylabel('Number of Elements')
    plt.title(f'Distribution of Clusters - {cluster_column}')
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()  # Ensures everything fits in the plot without overlapping
    plt.show()

    return summary_df

def find_optimal_clusters(data, key, approach, file_path, k_min=2, k_max=10, step=1):
    """
    Finds the optimal number of clusters by evaluating clustering performance across a range of cluster counts.

    This function iteratively applies a clustering algorithm (either KMeans or KMedoids) for different values of k (number of clusters).
    For each k, it calculates the sum of squared distances (SSE), Silhouette scores, and Davies-Bouldin scores. The results are then
    plotted to help identify the optimal number of clusters (the 'elbow point' in the SSE plot and optimal points in Silhouette and Davies-Bouldin score plots).

    Args:
        data (pandas.DataFrame): The DataFrame containing the data to be clustered.
        key (str): The key in the DataFrame that holds the data for clustering.
        approach (str): The clustering approach to use ('kmeans' or 'kmedoids').
        k_min (int): The minimum number of clusters to test.
        k_max (int): The maximum number of clusters to test.
        step (int): The step size to use when iterating over the range of cluster counts.
    """
    sse = []  # Sum of squared distances
    silhouette_scores = []
    bouldin_scores = []
    aggregated_scores = []
    k_values = range(k_min, k_max + 1, step)

    for k in tqdm(k_values):
        if approach == 'kmeans':
            parameters = {'n_clusters':k, 'init':'k-means++', 'n_init':'auto'}
        else:
            parameters = {'n_clusters': k, 'init': 'k-medoids++', 'metric':'cosine'}
        model, silhouette, davies_bouldin = cluster_and_evaluate(data, parameters, key, approach)

        # Calculate the sum of squared distances
        sse.append(model.inertia_)
        
        # Calculate the Silhouette score and Davies-Bouldin score
        silhouette_scores.append(silhouette)
        bouldin_scores.append(davies_bouldin)
        aggregated_scores.append(np.max(davies_bouldin - silhouette, 0))

    # Plotting the elbow curve and the scores
    plt.figure(figsize=(20, 20))
    plt.title(f'{key}_{approach}_{k_min}{k_max}{step}')

    plt.subplot(2, 2, 1)
    plt.plot(k_values, sse, '-o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Sum of squared distances (SSE)')
    plt.title('Elbow Curve for Inertia')
    plt.grid(True)

    plt.subplot(2, 2, 2)
    plt.plot(k_values, silhouette_scores, '-o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score Curve')
    plt.grid(True)

    plt.subplot(2, 2, 3)
    plt.plot(k_values, bouldin_scores, '-o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Davies-Bouldin Score')
    plt.title('Davies-Bouldin Score Curve')
    plt.grid(True)

    plt.subplot(2, 2, 4)
    plt.plot(k_values, aggregated_scores, '-o')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Aggregated Score')
    plt.title('Aggregated Score Curve')
    plt.grid(True)

    plt.tight_layout()

    if file_path:
        plt.savefig(file_path)
    
    plt.show()

def grid_search(parameter_combinations, data, key, approach, plot_enabled=True):
    """
    Performs a grid search to find the best clustering parameters based on evaluation metrics.

    This function iterates over a set of parameter combinations for a clustering algorithm, evaluating each combination
    using the cluster_and_evaluate function. It determines the best parameter set based on the Silhouette and Davies-Bouldin scores.
    The best configuration is determined based on the highest aggregated score, which is a combination of normalized Silhouette and Davies-Bouldin scores.

    Args:
        parameter_combinations (list of dicts): A list of dictionaries, each representing a set of clustering parameters.
        data (pandas.DataFrame): The DataFrame containing the data to cluster.
        key (str): The key in the DataFrame that contains the data to use for clustering.
        approach (str): The clustering approach to use ('kmeans' or 'kmedoids').
        plot_enabled (bool): If True, plots the clustering summary and vector elements for each parameter set.

    Returns:
        tuple: A tuple containing the best aggregated score and the corresponding best parameter configuration.

    """
    best_score = (0, 0, 0)  # Initialize the best score
    best_config = None  # Initialize the best configuration

    # Iterate over parameter combinations and evaluate each
    pbar = tqdm(enumerate(parameter_combinations), total=len(parameter_combinations))
    for i, params in pbar:
        pbar.set_description(f'Best score: {round(best_score[0],3)} - S:{round(best_score[1],3)} - B:{round(best_score[2],3)}')
        print(f'\nConfiguration {i+1}/{len(parameter_combinations)}')
        _, silhouette, davies_bouldin = cluster_and_evaluate(data, params, key, approach)

        # Normalize the Silhouette and Davies-Bouldin scores for combination
        normalized_silhouette = (silhouette + 1) / 2  # Normalize between 0 and 1
        normalized_davies_bouldin = 1 / (1 + davies_bouldin)  # Higher is better

        # Combine scores to determine the overall performance of the configuration
        aggregated_score = normalized_silhouette + normalized_davies_bouldin

        print(f'Parameters: {params}')
        print(f'Silhouette score: {silhouette}')
        print(f'Davies-Bouldin score: {davies_bouldin}')

        # Plot the clustering summary and vector elements if enabled
        if plot_enabled:
            clustering_summary(data, f'cluster_{approach}')
            plot_vector_elements(data, key, f'cluster_{approach}')

        # Update the best score and configuration if the current one is better
        if aggregated_score > best_score[0]:
            best_score = (aggregated_score, silhouette, davies_bouldin)
            best_config = params
    
    return (best_score, best_config)


### Routes generator with clustering - Task 1

In [ ]:
def transform_values(values, vocabulary):
    """
    Transforms and normalizes an array of values based on a given vocabulary's maximum quantity (max_qty).

    The function performs three main operations:
    1. Re-normalizes the values by scaling them according to the 'max_qty' from the vocabulary.
    2. Applies a threshold by setting any value less than 1 to zero, filtering out negligible values.
    3. Rounds the values to the nearest integer, converting them into integer format.

    Args:
        values (array): Array of values to be transformed.
        vocabulary (Vocabulary object): The vocabulary object.

    Returns:
        array: The transformed and normalized array of values as integers.
    """
    values = values * vocabulary.max_qty # 1
    values = np.where(values < 1, 0, values) # 2
    return np.round(values).astype(int) # 3


def create_new_routes(df, column_name, vocabulary, cluster_label):
    """
    Creates new routes for each cluster based on the average vector representations in the DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame containing the cluster data.
        column_name (str): The name of the column containing the vector representations of routes.
        vocabulary (Vocabulary object): The vocabulary object.
        cluster_label (str): The name of the column containing the cluster labels.

    Returns:
        list: A list of dictionaries, each representing a new route for a cluster.

    The function iterates over each cluster, calculates the mean vector for the routes in that cluster.
    For each mean vector constructs a new route by identifying the most significant city pairs and merchandise for each trip rapresented in the mean vector.
    The merchandise vector's part is decoded using transform_values() function above.
    """
    new_routes = []
    
    city_encoded_length = len(vocabulary.city2id)
    merch_encoded_length = len(vocabulary.merchandise2id)
    trip_vector_len = city_encoded_length + merch_encoded_length
    max_trip = vocabulary.max_trip

    # Iterate over each unique cluster
    for cluster in df[cluster_label].unique():
        # Calculate the mean vector for the routes in the cluster
        mean_vector = df[df[cluster_label] == cluster][column_name].mean()
        trips = []
        previous_start_index = -1

        # Construct each trip in the route
        for trip in range(max_trip):
            low_limit = trip * trip_vector_len
            trip_upper_limit = (trip + 1) * trip_vector_len

            # Initialize variables to find starting and ending cities
            city_start_max_v = 0
            city_start_max_v_index = 0 if previous_start_index == -1 else previous_start_index
            city_end_max_v = 0
            city_end_max_v_index = 0

            # Iterate over city segment of the mean vector
            for i, cell in enumerate(mean_vector[low_limit : trip_upper_limit - merch_encoded_length]):
                if cell != 0 and i < city_encoded_length:
                    if previous_start_index == -1 and cell > city_start_max_v:
                        city_start_max_v = cell
                        city_start_max_v_index = i
                    elif cell > city_end_max_v:
                        city_end_max_v = cell
                        city_end_max_v_index = i

            # Break if the start and end cities are the same
            if city_start_max_v_index == city_end_max_v_index:
                break

            # Process merchandise segment of the mean vector
            merch_decoded_vector = transform_values(mean_vector[trip_upper_limit - merch_encoded_length  : trip_upper_limit], vocabulary)
            merchandise = {}
            for i, cell in enumerate(merch_decoded_vector):
                if cell != 0:
                    merchandise[vocabulary.id2merchandise[i]] = cell

            # Create a trip if merchandise is found
            if merchandise:
                trip = {
                    'from' : vocabulary.id2city.get(city_start_max_v_index),
                    'to' : vocabulary.id2city.get(city_end_max_v_index),
                    'merchandise' : merchandise
                }
                previous_start_index = city_end_max_v_index
                trips.append(trip)

        # Add the constructed route for the cluster
        new_routes.append({
            'id' : f's{cluster}',
            'route': trips
        })

    return new_routes



### Driver favourite routes - Task 2

In [ ]:
def get_drivers_favourite_routes(merged_df, key_field, top_n_routes=10):
    """
    Computes and lists the favorite routes of each driver based on merged route data.

    Args:
        merged_df (pandas.DataFrame): DataFrame containing merged actual routes with the standard assigned.
        key_field (str): The name of the field in the DataFrame that holds the similarity score.
        top_n_routes (int): The number of top routes to list for each driver.

    Returns:
        pandas.DataFrame: A DataFrame where each row represents a driver and includes their average similarity score, 
                          the number of routes assigned and their top N routes based on preference scores.

    The function processes the merged route data to calculate the average similarity for each driver and identifies their 
    top N routes based on a calculated preference score. The preference score is a product of the average similarity and 
    the normalized count (frequency) of each standard route being assigned to the driver.
    """
    unique_drivers = merged_df['driver'].unique()
    driver_profiles = []

    for driver in sorted(unique_drivers):
        driver_df = merged_df[merged_df['driver'] == driver]

        # Calculate the average similarity score for each driver
        average_similarity = driver_df[key_field].mean()
        
        # Aggregate statistics for each standard route for the driver
        std_route_stats = driver_df.groupby('sroute_id')[key_field].agg(['mean', 'count'])

        # Normalize the count by the maximum count to get a relative frequency
        max_count = std_route_stats['count'].max()
        std_route_stats['normalized_count'] = std_route_stats['count'] / max_count
        
        # Calculate the preference score for each route
        std_route_stats['preference_score'] = std_route_stats['mean'] * std_route_stats['normalized_count']

        # Get the top N routes based on preference score
        top_routes = std_route_stats.sort_values(by='preference_score', ascending=False)[:top_n_routes]

        # Append the driver's profile including their average similarity and top N routes
        driver_profiles.append({
            'driver': driver, 
            'Routes assigned': len(std_route_stats),
            'average_similarity': average_similarity, 
            'top_routes': top_routes
        })

    # Return a DataFrame of all driver profiles, indexed by driver
    return pd.DataFrame(driver_profiles).set_index('driver')


### Route generation with collaborative filtering - Task 3

In [ ]:
def generate_optimal_route(matrix, vocabulary):
    """
    Generates optimal routes for each driver based on a rating matrix and vocabulary using a greedy approach.

    Args:
        matrix (numpy.ndarray): A matrix where each element represents a rating for a driver-trip pair.
        vocabulary (Vocabulary object): An object containing mapping and trip information.

    Returns:
        dict: A dictionary where each key is a driver index and the value is a route with the optimal trips.

    This function iterates over each driver and builds an optimal route by greedily selecting the best available trip at each step.
    The route construction stops if the maximum trip length is reached.
    """

    max_trip_length = vocabulary.max_trip
    num_drivers = matrix.shape[0]
    optimal_routes = {}

    for driver in range(num_drivers):
        current_city = None
        route = []
        trip_indices = list(range(matrix.shape[1]))

        while len(route) < max_trip_length:
            best_trip = None
            best_rating = -1

            for trip_index in trip_indices:
                trip_values = vocabulary.id2trip.get(trip_index)
                start_city, end_city = trip_values[0], trip_values[1]

                if current_city is None or current_city == start_city:
                    trip_rating = matrix[driver, trip_index]
                    if trip_rating > best_rating:
                        best_rating = trip_rating
                        best_trip = trip_index

            if best_trip is not None:
                trip_indices.remove(best_trip)
                route.append(vocabulary.id2trip.get(best_trip))
                current_city = vocabulary.id2trip.get(best_trip)[1]
            else:
                break

        optimal_routes[driver] = route

    return optimal_routes

def convert_routes_to_readable(optimal_routes, vocabulary):
    """
    Converts optimal routes into the requested format.

    Args:
        optimal_routes (dict): Dictionary containing optimal routes for each driver.
        vocabulary (Vocabulary object): An object containing mapping and trip information.

    Returns:
        list: A list where each element is a dictionary representing a driver's route in a readable format.
    """
    readable_routes = []
    for driver, route in optimal_routes.items():
        new_route = {'driver': vocabulary.id2driver[driver], 'route': []}

        for trip in route:
            new_trip = {'from': trip[0], 'to': trip[1], 'merchandise': {merch: qty for merch, qty in trip[2:]}}
            new_route['route'].append(new_trip)

        readable_routes.append(new_route)
    return readable_routes

## Main

### 1 - Load datasets and components

#### Vocabulary & Encoder

In [ ]:
DATASET_SIZE = '20_standard_1k_variations' # 50_standard_5k_variations, 50_standard_10k_variations, 20_standard_1k_variations, 20_standard_5k_variations, 20_standard_10k_variations
STANDARD_ROUTES_PATH = f'data/{DATASET_SIZE}/standard.json'
ACTUAL_ROUTES_PATH = f'data/{DATASET_SIZE}/actual.json'
N_PERMUTATIONS = 256

vocabulary = Vocabulary(STANDARD_ROUTES_PATH, ACTUAL_ROUTES_PATH)
encoder = Encoder(vocabulary, N_PERMUTATIONS)

print(f' - Vocabulary info:')
print(f' - Max number of consecutive trips: {vocabulary.max_trip}')
print(f' - Max quantity of a merch: {vocabulary.max_qty}')
print(f' - Min quantity of a merch: {vocabulary.min_qty}')
print(f'\n - Vocabularies print format (element, id)')
print(f' - {len(vocabulary.city2id)} cities - Example: {list(vocabulary.city2id.items())[1]}')
print(f' - {len(vocabulary.merchandise2id)} merchandise types - Example: {list(vocabulary.merchandise2id.items())[1]}')
print(f' - {len(vocabulary.trip2id)} Trips - Example: {list(vocabulary.trip2id.items())[1]}')
print(f' - {len(vocabulary.trip_city_pair2id)} trip city pair ids - Example: {list(vocabulary.trip_city_pair2id.items())[1]}')


#### Standard routes & actual routes

Loading standard and actual routes from files and the build a dataframe with:
- Matrix of the route - Matrix rapresentation of each route (shape Max trip X cities + merchandise_types)
- Vector of the route (flatten matrix)
- Vocabulary encoding of the route, cities and merch are translated to ids.
- Raw route and raw list of trips
- List of trip hashable tuples
- List of cities

In [ ]:
standard_routes = load_routes(STANDARD_ROUTES_PATH, vocabulary, encoder, 'standard')
actual_routes = load_routes(ACTUAL_ROUTES_PATH, vocabulary, encoder, 'actual')

merged_routes = pd.merge(actual_routes, standard_routes, on='sroute_id', how='inner', suffixes=( '_act', '_std'))

example_row = merged_routes[:2]
printTable(example_row[['aroute_id','sroute_id','driver','route_act','route_std','cities_act','cities_std']], 'First two rows of merged routes:\n')
printTable(example_row[['aroute_id','sroute_id','driver','trips_act','trips_std']], 'First two rows of merged routes:\n')
printTable(example_row[['aroute_id','sroute_id','driver','encoded_route_act','encoded_route_std']], 'First two rows of merged routes:\n')
printTable(example_row[['aroute_id','sroute_id','driver','matrix_act','matrix_std']], 'First two rows of merged routes:\n')
printTable(example_row[['aroute_id','sroute_id','driver','vector_act','vector_std']], 'First two rows of merged routes:\n')

### 2 - Utility matrix and collaborative trip filtering

#### Level 1 utility matrix

Generate the utility matrix at trip level. A single trip performed by a specific driver, as a matrix, is evaluated as follow:
- Examine an actual route matrix with the respective assigned standard route matrix.
- Process at trip level (row) the actual route.
- For each tripA (row) of actual route matrix consider each tripS (row) in the standard route matrix.
- (A) If tripA has an equal vector in the same position of the tripS, this is a complete match. (rating = 1)
- (B) Otherwise, a metric is used to compuet the similarity between tripA and each, non-zero (requested), tripS in the standard route matrix, this value is added to the summation matrix.
- (B) Then the number of counts, 1 in case (A), k in case (B), is saved in another matrix, the count matrix.

This process build two matrices: 
- one with the summation of similarity of each not correct trip with each trip requested in the correspondant standard route, 1 only for correct trips.
- one with the count of the comparison made for each trip.

The utility matrix is at the end computed as the summation matrix divided by the count matrix. This give the ratings of each performed trip for each driver.

In [ ]:
KEY = 'vector'
utility_matrix = UtilityMatrix(merged_routes, KEY, vocabulary)

print(f'Utility matrix level 1 loaded.')
print(f' - Shape: {utility_matrix.matrix_lv1.shape}')
print(f' - Max value: {utility_matrix.matrix_lv1.max()}')
print(f' - Min value: {utility_matrix.matrix_lv1.min()}')
print(f' - Mean value: {utility_matrix.matrix_lv1.mean()}')

#### Generate driver profiles

In [ ]:
profiles = DriversProfile(utility_matrix.matrix_lv1, vocabulary)

print(f'Driver profiles generated from level 1 utility matrix.')
print(f' - Shape: {profiles.drivers_profile.shape}')
print(f' - Max value: {profiles.drivers_profile.max()}')
print(f' - Min value: {profiles.drivers_profile.min()}')
print(f' - Mean value: {profiles.drivers_profile.mean()}')

#### Level 2 utility matrix - Collaborative filetring predictions with driver profiles

In [ ]:
# Define the value of k for drivers and trips
k_drivers = 100

# Compute top-k similar drivers
top_k_drivers = get_top_k_similar_indices(profiles.drivers_profile, k_drivers)

utility_matrix.predict_ratings_with_driver_profile_top_k(profiles.drivers_profile, top_k_drivers)

print(f'Utility matrix level 1 updated using collaborative filtering -> Level 2 matrix')
print(f' - Shape: {utility_matrix.matrix_lv2.shape}')
print(f' - Max value: {utility_matrix.matrix_lv2.max()}')
print(f' - Min value: {utility_matrix.matrix_lv2.min()}')
print(f' - Mean value: {utility_matrix.matrix_lv2.mean()}')

##### Plot matrices

In [ ]:
plot_matrix(utility_matrix.matrix_lv1, max_x=50, max_y=50, ylabel='Driver', xlabel='Trip', title='Utility matrix (Drivers x Trips) (Limited to 50x50)')
plot_matrix(profiles.drivers_profile, max_x=50, max_y=50, ylabel='Driver', xlabel='Trip', title='Drivers profiles(Drivers x Trips) (Limited to 50x50)')
plot_matrix(utility_matrix.matrix_lv2, max_x=50, max_y=50, ylabel='Driver', xlabel='Trip', title='Level 2 utility matrix with driver profile (Drivers x Trips) (Limited to 50x50)')

### 3 - Clustering

#### Trip ranking & trip rating feature

Using the previously computed utility matrix, here is computed a score for each trip as follow:
- Summed the non-zero values in each column (trip) of the utility matrix.
- Count of the non-zero trips for each column.
- (A) Compute the average of these scores.
- Get the total number of times that each trip has been executed
- (B) Normalize the previous step counts dividing by the maximum
- Compute the normalized score multipling the score at (A) by the normalized count at (B)

These final scores are then concatenated at the end of each correspondant row (trip) of the matrix of each route. 
This result in an increase of the dimensionality of 1.

The new matrices and vectors are inserted in two new columsn of the dataframe (actual route and standard routes):
- vector -> vector_f1
- matrix -> matrix_f1

In [ ]:
trip_ranking = trip_evaluation(utility_matrix.matrix_lv2, utility_matrix.trip_count_matrix, vocabulary)

add_trip_level_feature(trip_ranking, actual_routes, vocabulary, feature_name = 'normalized_score')
add_trip_level_feature(trip_ranking, standard_routes, vocabulary, feature_name = 'normalized_score')

printTable(trip_ranking, 'Top 10 trip ratings')
printTable(actual_routes[['matrix','matrix_f1']][:2], '\nRoute encode without and with the trip rating feature.')

#### Vector principal component analysis (PCA)

The followign trasformation will modifiy and add new columns to the standard routes and actual routes dataframes as follow:

- vector    -> vector_pca 
- vector_f1 -> vector_f1_pca

The left field is transformed with PCA to a reduced dimensionality for clustering analysis. The chosen dimensionality is 3.

In [ ]:
PCA_COMPONENTS = len(vocabulary.stdroutes)

# Generate vector_f1_pca
encoder.apply_PCA(actual_routes, 'vector_f1', PCA_COMPONENTS)
encoder.apply_PCA(standard_routes, 'vector_f1', PCA_COMPONENTS)

##### Routes in 2D and 3D

For plotting, the input vectors are reduced using PCA to be in 2D and 3D, if the vector shape doesn't match these two dimensionality constraints.

For example, this cause that for the normalized vector with PCA already applied, if the number of components is different from 3, the PCA is applied again over the previous PCA.

In [ ]:
plot_routes(actual_routes, 'vector')
plot_routes(actual_routes, 'vector_f1')
plot_routes(actual_routes, 'vector_f1_pca')

#### Number of clusters analysis - Results in /data/cluster_number_analysis/

This analysis will evaluate the numbers of cluster in the specified range ([x,y,step]).

Clusters are evaluated using the inertia, Silhouette score and Davis Bouldin score. The evaluation is performed on a basic configuration of the two model adopted: KMeans and KMedoids.

CLUSTERING_KEY define the type of encoding of the route to use for the clustering phase, or in other word the name of the column of the dataframe to use, which are:

- vector

- vector_f1 

- vector_f1_pca

This analysis was conducted for each available CLUSTERING_KEY. The rapresentation that yields best results in terms of silhouette score and David Bouldin score is the one that uses vector_f1_pca.

 The range analyzed is [2,100] with a step of 2 clusters. The charts are saved inside "cluster_number_analysis" folder in the same directory of this notebook.


In [ ]:
find_optimal_clusters(actual_routes, 'vector_f1_pca', 'kmeans', f'data/cluster_number_analysis/{DATASET_SIZE}/kmeans_clustering_range21002.png', 2, 100, 2) 
find_optimal_clusters(actual_routes, 'vector_f1_pca', 'kmedoids', f'data/cluster_number_analysis/{DATASET_SIZE}/kmedoids_clustering_range21002.png', 2, 100, 2)

#### Grid search for clustering methods hyperparameters

With this grid search the goal is to find the best hyperparameters configuration for the two adopted clustering models (KMeans and KMedoids). 

It's also possible to specify different cluster numbers but they should be setted considering the previous analysis.

In [ ]:
# Common property
cluster_sizes = [50]
CLUSTERING_KEY = 'vector_f1_pca'

# KMEAN
kmeans_parameter_combinations = [
    
    {'n_clusters': n_clusters, 'tol': tol, 'n_init':n_init, 'init': init, 'algorithm':algorithm, 'max_iter':max_iter}

    for n_clusters in cluster_sizes
    for tol in [1e-4, 1e-5, 1e-3] # default: 1e-4
    for n_init in [10] # default:10
    for init in ['k-means++'] # default:k-means++
    for algorithm in ['lloyd'] # default:lloyd - elkan
    for max_iter in [300, 600, 900] # default:300
]

best_score_kmeans, best_config_kmeans = grid_search(kmeans_parameter_combinations, actual_routes, CLUSTERING_KEY, 'kmeans', plot_enabled = False)

# KMEDOIDS
kmedoids_parameter_combinations = [
    
    {'n_clusters': n_clusters,  'init':init,  'metric': metric, 'method':method, 'max_iter':max_iter}

    for n_clusters in cluster_sizes
    for init in ['k-medoids++','random','build'] # default:heuristic - random - build
    for metric in ['cosine'] # default:euclidean
    for method in ['alternate'] # default:alternate - pam (has no end)
    for max_iter in [300] # default:300
]

best_score_kmedoids, best_config_kmedoids = grid_search(kmedoids_parameter_combinations, actual_routes, CLUSTERING_KEY, 'kmedoids', plot_enabled = False)

# Best configurations
print(f'\n\nGrid search analysis for KMeans\nBest configuration:\n - Silhouette score: {best_score_kmeans[1]}\n - Bouldin score: {best_score_kmeans[2]}\n - Hyperparameters:')
for key, el in best_config_kmeans.items():
    print(f' -- {key} : {el}')

print(f'\n\nGrid search analysis for KMedoids\nBest configuration:\n - Silhouette score: {best_score_kmedoids[1]}\n - Bouldin score: {best_score_kmedoids[2]}\n - Hyperparameters:')
for key, el in best_config_kmedoids.items():
    print(f' -- {key} : {el}')

#### Single configuration clustering

This is the code with fixed parameter for the two adopted clustering models (KMedoids and KMeans). 

The labels of the clusters are inserted into the appropriate column of the actual routes dataframe. Thus, each actual route has a cluster assigned.


In [ ]:
CLUSTERING_KEY = 'vector_f1_pca'

#KMEAN_CONFIGURATION = best_config_kmeans
KMEAN_CONFIGURATION = {'n_clusters': 50, 'init': 'k-means++', 'tol': 1e-4, 'algorithm':'lloyd', 'max_iter':300}

#KMEDOIDS_CONFIGURATION = best_config_kmedoids
KMEDOIDS_CONFIGURATION = {'n_clusters': 50, 'init': 'build', 'metric':'cosine', 'method':'alternate', 'max_iter':300}

kmeans_cluster_model, kmeans_silhouette, kmeans_davies_bouldin = cluster_and_evaluate(actual_routes, KMEAN_CONFIGURATION, CLUSTERING_KEY, approach='kmeans')
kmedoids_cluster_model, kmedoids_silhouette, kmedoids_davies_bouldin = cluster_and_evaluate(actual_routes, KMEDOIDS_CONFIGURATION, CLUSTERING_KEY, approach='kmedoids')

print(f'\nKMeans Parameters: {KMEAN_CONFIGURATION}')
print(f'Silhouette score: {kmeans_silhouette}')
print(f'Davies Bouldin score: {kmeans_davies_bouldin}')

print(f'\nKMedoids Parameters: {KMEDOIDS_CONFIGURATION}')
print(f'Silhouette score: {kmedoids_silhouette}')
print(f'Davies Bouldin score: {kmedoids_davies_bouldin}')

clustering_summary(actual_routes, cluster_column='cluster_kmeans')
clustering_summary(actual_routes, cluster_column='cluster_kmedoids')

##### Clusters in 2D and 3D

(For plotting, the input vectors are reduced using PCA to be in 2D and 3D, if the vector shape doesn't match these two dimensionality constraints)

In [ ]:
plot_vector_elements(actual_routes, CLUSTERING_KEY, 'cluster_kmeans')
plot_vector_elements(actual_routes, CLUSTERING_KEY, 'cluster_kmedoids')

### 4 - Clustering - Generate new standard route from centroids and medoids (Project task 1)

The new standard route with medoids corresponds to some actual routes.

The new standard routes with centroid are generated with the logic descibed in the function create_new_routes().

In [ ]:
REC_STANDARD_PATH = f'results/{DATASET_SIZE}/recStandard.json'

recStandard_from_mean = create_new_routes(actual_routes, 'vector', vocabulary, 'cluster_kmeans')
medoid = actual_routes.iloc[kmedoids_cluster_model.medoid_indices_]
recStandard_from_medoid = [{ 'id':f's{i}', 'route':route } for i, route in enumerate(medoid['route'])]

save_new_standard_routes(recStandard_from_mean + recStandard_from_medoid, REC_STANDARD_PATH)

printTable(medoid[['sroute_id','aroute_id','route']], 'Medoids:')
print(f'{r}\n' for r in recStandard_from_mean)


### 5 - Collaborative filtering - Generating new standard routes using the level 2 utility matrix (Project task 3)

Here are generated the ideal route for each driver. This route is build using the trip ratings in the level 2 utility matrix.

In [ ]:
OPTIMAL_DRIVERS_ROUTES_PATH = f'results/{DATASET_SIZE}/perfectRoute.json'
optimal_routes = generate_optimal_route(utility_matrix.matrix_lv2, vocabulary)
optimal_routes_encoded = convert_routes_to_readable(optimal_routes, vocabulary)
save_new_standard_routes(optimal_routes_encoded, OPTIMAL_DRIVERS_ROUTES_PATH)

for opt_route in optimal_routes_encoded:
    print(opt_route)

### 6 - Drivers best routes - Project task 2

#### Compute similarity scores of the merged routes

In [ ]:
rec_standard_routes = load_routes(REC_STANDARD_PATH, vocabulary, encoder, 'standard')
old_standard_routes = load_routes(STANDARD_ROUTES_PATH, vocabulary, encoder, 'standard')
new_standard_routes = pd.concat([rec_standard_routes, old_standard_routes], axis = 0, ignore_index = True)

merged_routes['jaccard_similarity'] = merged_routes.apply(lambda row: jaccard_similarity(row['signature_act'], row['signature_std']), axis=1)
merged_routes['cosine_similarity'] = merged_routes.apply(lambda row: cosine_similarity(row['vector_act'].reshape(1, -1), row['vector_std'].reshape(1, -1))[0,0], axis=1)
merged_routes['similarity'] = (merged_routes['jaccard_similarity'] + merged_routes['cosine_similarity']) / 2

#### Use the computed similarity score to analyze the driver route divergence

In [ ]:
DRIVER_PREFERENCES_PATH = f'results/{DATASET_SIZE}/driver.json'
driver_preferences = get_drivers_favourite_routes(merged_routes, 'similarity', top_n_routes=10)
save_top_routes_to_json(driver_preferences, DRIVER_PREFERENCES_PATH)

printTable(driver_preferences, 'Driver least divergence standard routes')

#### Drivers top-5 routes

In [ ]:
drivers_top_5_routes = print_driver_top_routes_from_json(DRIVER_PREFERENCES_PATH, standard_routes)
print(drivers_top_5_routes)

### 7 - Evaluation

To try to inspect the quality of the new standard routes with respect to the old one are used similarity matrices.
They are computed using the dataframe field specified in CLUSTERING_KEY for the cosine similarity matrix and the field signature for jaccard similarity matrices.

The scope is to observe how the new standard routes are distributed with respect to the actual and the old standard.


#### Old standard routes evaluation

In [ ]:
EVALUATION_KEY = 'vector'

jaccard_sm_std_act = calculate_similarity_matrix(standard_routes[:4], actual_routes[:4], 'signature', 'jaccard', f'../data/matrices/{DATASET_SIZE}/jaccard_sm_std_act.npy')
cosine_sm_std_act = calculate_similarity_matrix(standard_routes, actual_routes, EVALUATION_KEY, 'cosine', f'../data/matrices/{DATASET_SIZE}/cosine_sm_std_act.npy')

print(f'\nOld standard vs Actual - Jaccard similarity matrix:\n - Shape: {jaccard_sm_std_act.shape}\n - Mean: {jaccard_sm_std_act.mean().round(4)}\n - Std: {jaccard_sm_std_act.std().round(4)}')
print(f'\nOld standard vs Actual  - Cosine similarity matrix:\n - Shape: {cosine_sm_std_act.shape}\n - Mean: {cosine_sm_std_act.mean().round(4)}\n - Std: {cosine_sm_std_act.std().round(4)}')

#### New standard routes evaluation

In [ ]:
# New standard vs Actual
jaccard_sm_recstd_act = calculate_similarity_matrix(rec_standard_routes, actual_routes, 'signature', 'jaccard', f'../data/matrices/{DATASET_SIZE}/jaccard_sm_recstd_act.npy')
cosine_sm_recstd_act = calculate_similarity_matrix(rec_standard_routes, actual_routes, EVALUATION_KEY, 'cosine', f'../data/matrices/{DATASET_SIZE}/cosine_sm_recstd_act.npy')

# New standard vs Actual
print(f'\nNew Standard vs Actual  - Jaccard similarity matrix:\n - Shape: {jaccard_sm_recstd_act.shape}\n - Mean: {jaccard_sm_recstd_act.mean().round(4)}\n - Std: {jaccard_sm_recstd_act.std().round(4)}')
print(f'\nNew Standard vs Actual - Cosine similarity matrix:\n - Shape: {cosine_sm_recstd_act.shape}\n - Mean: {cosine_sm_recstd_act.mean().round(4)}\n - Std: {cosine_sm_recstd_act.std().round(4)}')

#### Plot evaluation matrices

In [ ]:
# Old standard vs Actual
plot_matrix(jaccard_sm_std_act, max_x= 25, max_y=25, xlabel= 'Standard routes', ylabel= 'Actual routes', title = 'Jaccard similarity matrix - OLD Standard vs Actual')
plot_matrix(cosine_sm_std_act, max_x= 25, max_y=25, xlabel= 'Standard routes', ylabel= 'Actual routes', title = 'Cosine similarity matrix - OLD Standard vs Actual')

# New standard vs Actual
plot_matrix(jaccard_sm_recstd_act, max_x= 25, max_y=25, xlabel= 'Rec Standard routes', ylabel= 'Actual routes', title = 'Jaccard similarity matrix - New Standard vs Actual')
plot_matrix(cosine_sm_recstd_act, max_x= 25, max_y=25, xlabel= 'Rec Standard routes', ylabel= 'Actual routes', title = 'Cosine similarity matrix - New Standard vs Actual')


#### Evaluation of top-5 identified favourite routes

In [ ]:
def load_json(location='results/20_standard_3k_variations/driver.json'):
    
    with open(location) as f:
        data = json.load(f)

    return {item['driver']: item['routes'] for item in data}

def calculate_similarity(df):
    
    return df.apply(lambda row: jaccard_similarity(np.array(row['signature_act']), np.array(row['signature_std'])), axis=1).mean()

def process_driver_data(merged_routes, driver_prefix, driver_fav_routes):
    
    all_df = merged_routes[merged_routes['driver'] == f'{driver_prefix}']
    fav_df = merged_routes[(merged_routes['driver'] == f'{driver_prefix}') & (merged_routes['sroute_id'].isin(driver_fav_routes))]
    non_fav_df = merged_routes[(merged_routes['driver'] == f'{driver_prefix}') & (~merged_routes['sroute_id'].isin(driver_fav_routes))]
    
    a = calculate_similarity(fav_df)
    b = calculate_similarity(all_df)
    c = calculate_similarity(non_fav_df)
    
    return a, b, c

# Load JSON data
data_dict = load_json()

# Process data for each driver
fav_route_similarity_scores, all_routes_similarity_scores, non_fav_route_similarity_scores = zip(*[process_driver_data(merged_routes, f'driver_{i + 1}', data_dict[f'driver_{i+1}']) for i in range(len(data_dict.keys()))])

# Concatenate arrays into a single DataFrame
columns = ['Only favourite routes', 'All routes', 'Routes without favourites']
driver_values = pd.DataFrame(list(zip(fav_route_similarity_scores, all_routes_similarity_scores, non_fav_route_similarity_scores)), columns=columns, index=[f'driver_{i + 1}' for i in range(len(data_dict.keys()))])

# Plotting
plot_similarity(driver_values)
driver_values.describe() # the statistics
driver_values.mean() # aggregated mean, showing the overall increase


## Info

In [ ]:
# Vocabulary info

print(f' - Vocabulary max qty:', vocabulary.max_qty)
print(f' - Dataset, loaded {len(standard_routes)} standard routes.')
print(f' - Dataset, loaded {len(actual_routes)} actual routes.')
print(f' - Dataset, loaded {len(merged_routes)} merged routes.')
print(f' - Vector len: ', len(actual_routes['vector'][0]))
print(f' - Matrix shape: ', actual_routes['matrix'][0].shape)
print(f' - Signature len: ', len(actual_routes['signature'][0]))

print(f'\nDataframes structure (standard and actual dataframes have same structure):\n')
print(standard_routes.info(), '\n\n')
print(merged_routes.info(), '\n\n')

# Actual routes
print('\nActual routes examples:')
printTable(actual_routes[['aroute_id', 'route']][2:3], 'Route:')
printTable(actual_routes[['aroute_id', 'cities']][2:3], 'Cities:')
printTable(actual_routes[['aroute_id', 'trips']][2:3], 'Trips:')
printTable(actual_routes[['aroute_id', 'encoded_route']][2:3], 'Encoded route:')
printTable(actual_routes[['aroute_id', 'matrix']][2:3], 'Route matrix:')
printTable(actual_routes[['aroute_id', 'matrix_f1']][2:3], 'Route matrix:')
printTable(actual_routes[['aroute_id', 'signature']][2:3], 'Signature:')



print(f'jaccard_similarity   - MAX:',merged_routes['jaccard_similarity'].max(),' - MIN:', merged_routes['jaccard_similarity'].min(), ' - MEAN:',merged_routes['jaccard_similarity'].mean())
print(f'cosine_similarity    - MAX:',merged_routes['cosine_similarity'].max(),' - MIN:', merged_routes['cosine_similarity'].min(), ' - MEAN:',merged_routes['cosine_similarity'].mean())
print(f'euclidean_similarity - MAX:',merged_routes['euclidean_similarity'].max(),' - MIN:', merged_routes['euclidean_similarity'].min(), ' - MEAN:',merged_routes['euclidean_similarity'].mean())

printTable(merged_routes[[
    'aroute_id', 'sroute_id','cities_std','cities_act',
    'jaccard_similarity',
    'cosine_similarity',
    'euclidean_similarity',
]][:100], 'Similarities:', limit=100)